## Import librairies

In [454]:
import sys
from pathlib import Path
import os
import json
import pandas as pd
import numpy as np
import seaborn as sns
import os
import locale
import matplotlib
import matplotlib.pyplot as plt
from datetime import datetime,timedelta

from sklearn.preprocessing import MultiLabelBinarizer

# Ajouter le dossier parent src au path
sys.path.append(os.path.abspath('..'))

%matplotlib inline


## CONSTANTES

In [455]:
EXTRACT_DATE = datetime(2025, 11, 1, 17, 2, 28)

PRICE_SEGMENTS = [
    {
        "label": "0 - 7.99",
        "value_min": 0,
        "value_max": 7.99,
    },
    {
        "label": "8 - 14.99",
        "value_min": 8,
        "value_max": 14.99,
    },
    {
        "label": "15 - 24.99",
        "value_min": 15,
        "value_max": 24.99,
    },
    {
        "label": "25 - 39.99",
        "value_min": 25,
        "value_max": 39.99,
    },
    {
        "label": "40+",
        "value_min": 40,
        "value_max": 150,
    },
]

WAIT_DISCOUNT_SEGMENTS = [
    {
        "label": "0-3_mois",
        "value_min": 0,
        "value_max": 90,
    },
    {
        "label": "3-6_mois",
        "value_min": 91,
        "value_max": 180,
    },
    {
        "label": "6-12_mois",
        "value_min": 181,
        "value_max": 365,
    },
    {
        "label": "1-2_ans",
        "value_min": 366,
        "value_max": 730,
    },
    {
        "label": "jamais_ou_plus",
        "value_min": 731,
        "value_max": float('inf'),
    },
]

PROMOS = [10,25,33,50,75]


## Chargement des données

In [456]:
# Fichier stocké temporairement sur un de mes S3
# url = "https://d3aok2axxchxf9.cloudfront.net/ia/games_data.csv"

url = os.path.join(Path.cwd().parent, "data/processed/games_data.csv")

# Charger les données CSV, Les colonnes Boolean sont au format Int64 (0,1,NaN)
df = pd.read_csv(url,dtype={
    'trophies_count': 'Int64',
    'local_multiplayer_max_players': 'Int64',
    'online_multiplayer_max_players': 'Int64',
    'difficulty': 'Int64',
    'download_size': 'Int64',
    'hours_main_story': 'Int64',
    'hours_completionist': 'Int64',
    'metacritic_critic_score': 'Int64',
    'metacritic_critic_userscore': 'Int64',
    'pegi_rating': 'Int64',
})


## Vérification des données chargées

In [457]:
def column_summary(df: pd.DataFrame):
    summary = []
    for col in df.columns:
        col_type = df[col].dtype
        non_null = df[col].notna().sum()
        null_count = df[col].isna().sum()
        
        # Gérer le cas où la colonne contient des listes (unhashable)
        try:
            unique_count = df[col].nunique()
        except TypeError:
            # Si erreur (listes), convertir en string temporairement
            unique_count = df[col].astype(str).nunique()
            print(f"⚠️ Colonne '{col}' contient des types non-hashable (probablement des listes)")

        summary.append({
            'Column': col,
            'Type': str(col_type),
            'Non-Null Count': non_null,
            'Null Count': null_count,
            'Unique Values': unique_count,
        })

    # Afficher le résumé des colonnes
    print("=" * 80)
    print("Résumé détaillé des colonnes:")
    print("=" * 80)
    column_summary_df = pd.DataFrame(summary)
    print(column_summary_df.to_string(index=False))
    print("\n")

In [458]:
column_summary(df)

Résumé détaillé des colonnes:
                        Column    Type  Non-Null Count  Null Count  Unique Values
                     game_name  object            5382           0           5382
                      id_store  object            5382           0           5382
                short_url_name  object            5382           0           5382
                     publisher  object            5382           0           1083
                     developer  object            3920        1462           2709
                  release_date  object            5382           0           1347
         pssstore_stars_rating float64            5382           0            352
   pssstore_stars_rating_count   int64            5382           0           1583
       metacritic_critic_score   Int64            1268        4114             67
   metacritic_critic_userscore   Int64            1268        4114             10
                        genres  object            5360          22  

In [459]:
df.head()

,game_name,id_store,short_url_name,publisher,developer,release_date,pssstore_stars_rating,pssstore_stars_rating_count,metacritic_critic_score,metacritic_critic_userscore,...,hours_main_story,hours_completionist,pegi_rating,esrb_rating,rating_descriptions,voice_languages,subtitle_languages,base_price,lowest_price,price_history
0,"#Halloween, Super Puzzles Dream",EP8311-PPSA19174_00-0421646910657705,game-halloween-super-puzzles-dream-ps5,e-llusiontertainment,NaN,2023-10-30,1.57,14,<NA>,<NA>,...,<NA>,<NA>,7,Everyone,Fear/Horror,NaN,NaN,4.99,1.49,"[{""x"": ""2023-11-02"", ""y"": 4.99}, {""x"": ""2023-1..."
1,#SinucaAttack,EP2005-PPSA06055_00-SINUCA0000000000,game-sinucaattack-ps5,eastasiasoft,NaN,2022-05-19,3.26,72,<NA>,<NA>,...,0,0,3,Everyone,Fantasy Violence,NaN,NaN,4.99,1.49,"[{""x"": ""2022-05-21"", ""y"": 4.99}, {""x"": ""2022-1..."
2,"#Wish travel, super puzzle dreams",EP8311-PPSA16513_00-0233078860249892,game-wish-travel-super-puzzle-dreams-ps5,e-llusiontertainment,NaN,2023-05-31,1.55,11,<NA>,<NA>,...,<NA>,<NA>,3,Everyone,NaN,NaN,NaN,4.99,1.49,"[{""x"": ""2023-06-19"", ""y"": 4.99}, {""x"": ""2023-0..."
3,"#WomenUp, Super Puzzles Dream",EP8311-PPSA13840_00-0277389480637871,game-womenup-super-puzzles-dream-ps5,e-llusiontertainment,NaN,2023-01-26,1.44,18,<NA>,<NA>,...,<NA>,<NA>,3,Everyone,NaN,NaN,NaN,4.99,1.49,"[{""x"": ""2023-05-21"", ""y"": 4.99}, {""x"": ""2023-0..."
4,"#Xmas, Super Puzzles Dream",EP8311-PPSA12662_00-0212989199890961,game-xmas-super-puzzles-dream-ps5,e-llusiontertainment,NaN,2022-12-22,1.17,23,<NA>,<NA>,...,<NA>,<NA>,3,Everyone,NaN,NaN,NaN,4.99,1.49,"[{""x"": ""2023-01-23"", ""y"": 4.99}, {""x"": ""2023-0..."


## Methodes utils

In [460]:
def draw_binary_circular_plots(data: list, name: str, axe: plt.Axes):
    colors = sns.color_palette("crest")
    values = [item["value"] for item in data]
    labels = [item["label"] for item in data]

    wedges, texts, autotexts = axe.pie(
        values,
        labels=labels,
        colors=colors,
        autopct="%1.1f%%",
        startangle=90,
        shadow=False,
    )

    for autotext in autotexts:
        autotext.set_color("white")
        autotext.set_fontweight("bold")

    axe.set_title(f"{name}", fontweight="bold", pad=20)

In [461]:
def layout_plots(n_cols: int):
    # Calculer le nombre de lignes nécessaires
    actual_rows = (n_cols + 2) // 3  # 3 colonnes par ligne

    # Limiter à 3 colonnes maximum
    actual_cols = min(n_cols, 3)

    # Créer la figure avec plus d'espace vertical
    fig, axes = plt.subplots(
        actual_rows, 
        actual_cols, 
        figsize=(14, 5 * actual_rows),  # Hauteur dynamique selon le nombre de lignes
        gridspec_kw={'hspace': 0.5, 'wspace': 0.3}  # ← Espace entre les lignes/colonnes
    )

    # Normaliser axes pour toujours retourner un tableau 1D itérable
    if n_cols == 1:
        axes = [axes]
    elif actual_rows == 1 and actual_cols == 1:
        axes = [axes]
    elif actual_rows == 1:
        axes = list(axes)
    else:
        axes = list(axes.flatten())

    return fig, axes

In [462]:
def generate_multi_str_col_top_proportion_data(
    df: pd.DataFrame, col_name: str, top_count: int
):

    # Séparer et exploser
    col_exploded = df[col_name].str.split(",").explode()
    col_exploded = col_exploded.str.strip()
    col_value_count = col_exploded.value_counts()

    # Garder le top 6 et regrouper le reste dans "Autres"
    top_val = col_value_count.head(top_count)
    autres = col_value_count.iloc[top_count:].sum()

    # Créer les données finales
    if autres > 0:
        final_counts = pd.concat([top_val, pd.Series({"Autres": autres})])
    else:
        final_counts = top_val

    result = []
    for label, value in final_counts.items():
        result.append(
            {
                "label": label,
                "value": value,
            }
        )

    return result

In [463]:
def generate_binary_cols_proportion_data(df: pd.DataFrame, column_name: str):
    binary_cols = [col for col in df.columns if col == column_name]

    if len(binary_cols) == 0:
        return None

    result = [
        {
            "label": "True",
            "value": (
                (df[binary_cols[0]] == 1).sum() / len(df) * 100 if len(df) > 0 else 0
            ),
        },
        {
            "label": "False",
            "value": (
                (df[binary_cols[0]] == 0).sum() / len(df) * 100 if len(df) > 0 else 0
            ),
        },
    ]

    return result

In [464]:
df_featured = df.copy()

## Transformation des dates en valeures numérique

In [465]:
def transform_dates(df:pd.DataFrame,today:datetime):
    df['release_date_dt'] = pd.to_datetime(df['release_date'], errors='coerce')
    df['release_year'] = df['release_date_dt'].dt.year
    df['release_month'] = df['release_date_dt'].dt.month
    df['days_since_release'] = (today - df['release_date_dt']).dt.days
    
    df.drop(['release_date_dt'], axis='columns', inplace=True)
    return df

In [466]:
df_featured = transform_dates(df_featured,EXTRACT_DATE)

## Création des features délais avant promos depuis la sortie (à prédire)

In [467]:
df_days_to_discount = df_featured.copy()

In [468]:
# Convertir la colonne price_history de JSON vers list
df_days_to_discount['price_history'] = df_days_to_discount['price_history'].apply(
    lambda x: json.loads(x) if isinstance(x, str) else x
)

In [469]:
def days_until_first_discount(
    sales_history, base_price, release_date, discount_threshold=0
):
    if release_date is None:
        return None

    if len(sales_history) == 0:
        return None

    if not isinstance(base_price, (int, float)):
        return None

    # Convertir release_date en datetime si c'est une string
    if isinstance(release_date, str):
        release_date = datetime.strptime(release_date, "%Y-%m-%d")

    # Calculer le prix cible selon le seuil de réduction
    target_price = base_price * (1 - discount_threshold / 100)

    # Parcourir l'historique pour trouver la première baisse atteignant le seuil
    for entry in sales_history:
        price = entry["y"]
        date_str = entry["x"]
        date = datetime.strptime(date_str, "%Y-%m-%d")

        # Ignorer les prix négatifs
        if price < 0.1:
            continue

        # Vérifier si la réduction atteint le seuil et après la date de sortie
        if price <= target_price and date >= release_date:
            days_diff = (date - release_date).days
            return days_diff

    # Aucune baisse au seuil voulu trouvée
    return None

In [470]:
def generate_day_to_discount_df_data(df:pd.DataFrame):
    for promo in PROMOS:
        col_name = f'days_to_{promo}_percent_discount'
        df[col_name] = df.apply(
            lambda row: days_until_first_discount(
                row['price_history'], 
                row['base_price'], 
                row['release_date'], 
                promo
            ),
            axis=1
        )
    return df

In [471]:
df_days_to_discount = generate_day_to_discount_df_data(df_days_to_discount)

In [472]:
for promo in PROMOS:
    col_name = f'days_to_{promo}_percent_discount'
    print('\n')
    print(df_days_to_discount[col_name].describe())



count    4778.000000
mean      111.046463
std       114.792521
min         0.000000
25%        48.000000
50%        76.000000
75%       128.000000
max      1292.000000
Name: days_to_10_percent_discount, dtype: float64


count    4509.000000
mean      139.444666
std       130.566226
min         0.000000
25%        57.000000
50%        98.000000
75%       175.000000
max      1292.000000
Name: days_to_25_percent_discount, dtype: float64


count    3714.000000
mean      205.792138
std       159.093618
min         0.000000
25%        97.000000
50%       168.000000
75%       268.000000
max      1533.000000
Name: days_to_33_percent_discount, dtype: float64


count    3091.000000
mean      290.918149
std       207.714124
min         0.000000
25%       142.000000
50%       247.000000
75%       380.000000
max      1574.000000
Name: days_to_50_percent_discount, dtype: float64


count    1025.000000
mean      519.724878
std       324.242354
min         1.000000
25%       266.000000
50%       476

In [473]:
def day_to_promo_summary(df:pd.DataFrame):
    summary_data = []
    for promo in PROMOS:
        col = f'days_to_{promo}_percent_discount'
        summary_data.append({
            'Réduction': f'-{promo}%',
            'Jeux concernés': df[col].notna().sum(),
            'Pourcentage': f"{(df[col].notna().sum() / len(df)) * 100:.1f}%",
            'Médiane (jours)': df[col].median(),
            'Moyenne (jours)': df[col].mean(),
            'Min (jours)': df[col].min(),
            'Max (jours)': df[col].max()
        })

    summary_df = pd.DataFrame(summary_data)
    print("\n" + "="*80)
    print("RÉSUMÉ DES PROMOTIONS")
    print("="*80)
    print(summary_df.to_string(index=False))

In [474]:
day_to_promo_summary(df_days_to_discount)


RÉSUMÉ DES PROMOTIONS
Réduction  Jeux concernés Pourcentage  Médiane (jours)  Moyenne (jours)  Min (jours)  Max (jours)
     -10%            4778       88.8%             76.0       111.046463          0.0       1292.0
     -25%            4509       83.8%             98.0       139.444666          0.0       1292.0
     -33%            3714       69.0%            168.0       205.792138          0.0       1533.0
     -50%            3091       57.4%            247.0       290.918149          0.0       1574.0
     -75%            1025       19.0%            476.0       519.724878          1.0       1806.0


In [475]:
def create_wait_to_discount_category(df: pd.DataFrame, target_promo: int):
    df_copy = df.copy()
    target_col = f'days_to_{target_promo}_percent_discount'
    col_name = f'days_to_{target_promo}_percent_discount_category'
    
    # Extraire les bins et labels depuis WAIT_DISCOUNT_SEGMENTS
    bins = [segment['value_min'] for segment in WAIT_DISCOUNT_SEGMENTS] + [float('inf')]
    labels = [segment['label'] for segment in WAIT_DISCOUNT_SEGMENTS]
    
    # Créer les catégories
    df_copy[col_name] = pd.cut(
        df_copy[target_col],
        bins=bins,
        labels=labels,
        include_lowest=True,
        right=True
    )
    
    df_copy[col_name] = df_copy[col_name].astype(str)
    
    # Gérer les NaN pour days_since_release
    # le jeu n'a JAMAIS atteint une baisse de {target_promo}%
    mask_nan = df_copy[target_col].isna()
    
    # jeux récents (<2 ans) qui n'ont PAS ENCORE atteint {target_promo}%
    mask_recent = mask_nan & (df_copy['days_since_release'] < 730)
    df_copy.loc[mask_recent, col_name] = 'moins_de_2_ans_sans_baisse'
    
    # jeux anciens (>=2 ans) qui n'ont JAMAIS atteint {target_promo}%
    mask_old = mask_nan & (df_copy['days_since_release'] >= 730)
    df_copy.loc[mask_old, col_name] = 'jamais_ou_plus'
    
    # NaN dans days_since_release = jeu par encore sortis ou erreur de date de sortie
    mask_unknown = mask_nan & df_copy['days_since_release'].isna()
    df_copy.loc[mask_unknown, col_name] = 'moins_de_2_ans_sans_baisse'
    
    return df_copy

In [476]:
# Pour chaque seuil de promo on crée la colonne de catégorie associée
for promo in PROMOS:
    df_days_to_discount = create_wait_to_discount_category(df_days_to_discount, promo)

In [477]:
def verification_to_discount_categorie(df:pd.DataFrame):
    for promo in PROMOS:
        col_name = f'days_to_{promo}_percent_discount_category'
        print('\n')
        print(df_days_to_discount[col_name].value_counts())

In [478]:
verification_to_discount_categorie(df_days_to_discount)



days_to_10_percent_discount_category
0-3_mois                      2898
3-6_mois                      1161
moins_de_2_ans_sans_baisse     559
6-12_mois                      533
1-2_ans                        160
jamais_ou_plus                  71
Name: count, dtype: int64


days_to_25_percent_discount_category
0-3_mois                      2090
3-6_mois                      1349
moins_de_2_ans_sans_baisse     813
6-12_mois                      799
1-2_ans                        237
jamais_ou_plus                  94
Name: count, dtype: int64


days_to_33_percent_discount_category
moins_de_2_ans_sans_baisse    1367
6-12_mois                     1244
3-6_mois                      1145
0-3_mois                       854
1-2_ans                        414
jamais_ou_plus                 358
Name: count, dtype: int64


days_to_50_percent_discount_category
moins_de_2_ans_sans_baisse    1847
6-12_mois                     1216
1-2_ans                        702
3-6_mois                       

In [479]:
# concat dataframe features avec dataframe days to discount des nouvelles colonnes uniquement
for promo in PROMOS:
    col_name_regression = f'days_to_{promo}_percent_discount'
    df_featured = pd.concat([df_featured, df_days_to_discount[[col_name_regression]]], axis=1)
    col_name_classification = f'days_to_{promo}_percent_discount_category'
    df_featured = pd.concat([df_featured, df_days_to_discount[[col_name_classification]]], axis=1)

In [480]:
# On vérifie les colonnes ajoutées
column_summary(df_featured)

Résumé détaillé des colonnes:
                              Column    Type  Non-Null Count  Null Count  Unique Values
                           game_name  object            5382           0           5382
                            id_store  object            5382           0           5382
                      short_url_name  object            5382           0           5382
                           publisher  object            5382           0           1083
                           developer  object            3920        1462           2709
                        release_date  object            5382           0           1347
               pssstore_stars_rating float64            5382           0            352
         pssstore_stars_rating_count   int64            5382           0           1583
             metacritic_critic_score   Int64            1268        4114             67
         metacritic_critic_userscore   Int64            1268        4114             10
  

## Traitement éditeur / developer

In [518]:
df_editor_dev_importance = df_featured.copy()

### Création fichier avec valeur unique

In [519]:
# Créer le DataFrame avec les comptages et trier
publisher_counts = df_editor_dev_importance['publisher'].value_counts().reset_index()
publisher_counts.columns = ['publisher', 'count']

# Sauvegarder en CSV
publisher_counts.to_csv('publisher_counts.csv', index=False)

## Traitement de la colonne des genres

In [481]:
df_genres_proccess = df_featured.copy()

In [482]:
# Liste compléte des genres

# Action	
# Adventure	
# Arcade	
# Casual	
# FPS	
# Fighting	
# Horror	
# MMO	
# Music	
# Platformer	
# Puzzle	
# RPG	
# Racing	
# Simulation	
# Sports	
# Strategy	
# TPS


# Regroupement en genres principaux: 

# https://cnlj.bnf.fr/sites/default/files/cnlj-genres-jeux-video.pdf

# LES JEUX D’ACTION - AVENTURE : Action,FPS, TPS, Platformer, Adventure, Horror
# LES JEUX DE ROLES : MMO, RPG
# LES JEUX DE SPORTS : Sports, Racing, Fighting
# LES JEUX DE REFLEXION : Puzzle, Simulation, Strategy
# LES JEUX RAPIDE : Music, Casual, Arcade

# On supprime les colonnes inutiles
# df_genres_proccess.drop(['genres'], axis='columns', inplace=True)



In [483]:
# Configuration des regroupements de genres
GENRE_GROUPS = {
    'action_aventure': ['Action', 'FPS', 'TPS', 'Platformer', 'Adventure', 'Horror'],
    'roles': ['MMO', 'RPG'],
    'sports': ['Sports', 'Racing', 'Fighting'],
    'reflexion': ['Puzzle', 'Simulation', 'Strategy'],
    'rapide': ['Music', 'Casual', 'Arcade']
}

In [484]:
def create_grouped_genre_binary_columns(df: pd.DataFrame) -> pd.DataFrame:
    """
    Crée des colonnes binaires pour les groupes de genres
    Un jeu peut appartenir à plusieurs groupes
    """
    df_result = df.copy()
    
    # Préparer les genres (split et strip)
    genres_lists = df_result['genres'].fillna('').str.split(',').apply(
        lambda x: [genre.strip() for genre in x if genre.strip()]
    )
    
    # Créer les colonnes binaires pour chaque groupe
    for group_name, group_genres in GENRE_GROUPS.items():
        df_result[f'genre_{group_name}'] = genres_lists.apply(
            lambda genres: int(any(genre in group_genres for genre in genres))
        )
    
    genre_columns = [f'genre_{group}' for group in GENRE_GROUPS.keys()]
    genre_df = df_result[genre_columns].copy()
    
    # S'assurer que toutes les valeurs sont 0 ou 1
    genre_df = genre_df.fillna(0).astype(int)
    
    return genre_df

In [485]:
df_genres_grouped = create_grouped_genre_binary_columns(df_genres_proccess)

# Vérifier le résultat
print("\n✅ Colonnes créées:")
print(df_genres_grouped.columns.tolist())
print(f"\n📋 Aperçu:")
print(df_genres_grouped.head(10))


✅ Colonnes créées:
['genre_action_aventure', 'genre_roles', 'genre_sports', 'genre_reflexion', 'genre_rapide']

📋 Aperçu:
   genre_action_aventure  genre_roles  genre_sports  genre_reflexion  \
0                      0            0             0                1   
1                      1            0             0                1   
2                      0            0             0                1   
3                      0            0             0                1   
4                      0            0             0                1   
5                      1            0             0                1   
6                      0            0             0                1   
7                      0            0             0                1   
8                      0            0             0                1   
9                      1            0             0                0   

   genre_rapide  
0             0  
1             0  
2             0  
3           

In [486]:
# Stats
print("Distribution des groupes de genres:")
print("=" * 60)
for col in df_genres_grouped:
    count = df_genres_grouped[col].sum()
    pct = count / len(df_genres_grouped) * 100
    print(f"   {col:30s}: {count:5d} jeux ({pct:5.1f}%)")

# Vérifier les jeux multi-groupes
total_assignments = df_genres_grouped.sum(axis=1)
print(f"\nNombre de groupes par jeu:")
print(f"   Sans groupe: {(total_assignments == 0).sum()}")
print(f"   1 groupe: {(total_assignments == 1).sum()}")
print(f"   2 groupes: {(total_assignments == 2).sum()}")
print(f"   3+ groupes: {(total_assignments >= 3).sum()}")

Distribution des groupes de genres:
   genre_action_aventure         :  4005 jeux ( 74.4%)
   genre_roles                   :  1044 jeux ( 19.4%)
   genre_sports                  :   736 jeux ( 13.7%)
   genre_reflexion               :  2582 jeux ( 48.0%)
   genre_rapide                  :  2805 jeux ( 52.1%)

Nombre de groupes par jeu:
   Sans groupe: 22
   1 groupe: 1506
   2 groupes: 2202
   3+ groupes: 1652


In [487]:
# Merge avec df_featured
df_featured = pd.concat([df_featured, df_genres_grouped], axis=1)

In [488]:
column_summary(df_featured)

Résumé détaillé des colonnes:
                              Column    Type  Non-Null Count  Null Count  Unique Values
                           game_name  object            5382           0           5382
                            id_store  object            5382           0           5382
                      short_url_name  object            5382           0           5382
                           publisher  object            5382           0           1083
                           developer  object            3920        1462           2709
                        release_date  object            5382           0           1347
               pssstore_stars_rating float64            5382           0            352
         pssstore_stars_rating_count   int64            5382           0           1583
             metacritic_critic_score   Int64            1268        4114             67
         metacritic_critic_userscore   Int64            1268        4114             10
  

## Création des features d'interactions

In [489]:
def create_interaction_features(df:pd.DataFrame):
    """
    Créer des features d'interaction qui peuvent avoir plus de pouvoir prédictif
    """
    df_new = df.copy()
    
    # Interactions multiplicatives
    df_new['price_x_rating'] = df_new['base_price'] * df_new['pssstore_stars_rating'].fillna(0)
    df_new['price_x_popularity'] = df_new['base_price'] * np.log1p(df_new['pssstore_stars_rating_count'].fillna(0))
    
    # Ratios
    df_new['price_per_hour'] = df_new['base_price'] / (df_new['hours_main_story'].fillna(10) + 1)
    df_new['rating_per_review'] = df_new['pssstore_stars_rating'].fillna(0) / (df_new['pssstore_stars_rating_count'].fillna(1) + 1)
    
    # Score composite
    df_new['quality_score'] = (
        df_new['pssstore_stars_rating'].fillna(0) * 20 +
        df_new['metacritic_critic_score'].fillna(0)
    ) / 2
    
    # Serie count > 3 & Main genre Sport -> renouvellement
    
    # Nombre de jeux sortie par editeur avant release == editeur majeure
    
    # Score Jeux connus ou sortie dans l'indifférence
    #   prix très bas -= 3
    #   critic pro existe += 2
    #   nombre de dlc += 1
    #   microtransaction += 1
    #   éditeur studio majeures += 3
    #   infos de completion existe += 1
    #   infos de difficulté existe += 1
    #   pack deluxe existe += 1
    #   nombre de votes > 100 += 2
    #   voices renseigne += 1
    #   subs renseigne += 1
    #   trophy count non connus -= 1
    
    # Contenus riche / rejouabilité (à pondéré avec base price)
    # 
    # nombre de trophée 
    # taille disque élevé
    # has_online_multiplayer
    # has_local_multiplayer
    # sous titres count
    # voices count
    # completion duration
    
    # playstaion_exclu_content (is_ps_exclusive or is_opti_ps5_pro or is_vr)
    
    return df_new

In [490]:
# Créer les nouvelles features
df_interactions = create_interaction_features(df_featured)

## Score visibilité médiatisé / peu médiatisé

Combiner des signaux faibles en un score composite fort
Indicateurs négatifs jeu inconnu/peu médiatisé
Indicateurs positifs (jeu connu/médiatisé)

In [491]:
df_visibility_score = df_featured.copy()

In [492]:
def calculate_visibility_score(df: pd.DataFrame) -> pd.Series:
    """
    Score élevé = jeu connu, bien marketé, éditeur majeur
    Score faible = jeu inconnu, peu d'infos
    """
    
    # Configuration du scoring (à ajuster)
    VISIBILITY_SCORE_RULES = {
        # Indicateurs négatifs (jeu inconnu/peu médiatisé)
        'very_low_price': {'condition': lambda df: df['base_price'] < 4.9, 'points': -2},
        'no_trophy_info': {'condition': lambda df: df['trophies_count'].isna(), 'points': -1},
        'has_no_reviews': {'condition': lambda df: df['pssstore_stars_rating_count'] < 10, 'points': -1},
        
        # Indicateurs positifs (jeu connu/médiatisé)
        'has_critic_score': {'condition': lambda df: df['metacritic_critic_score'].notna(), 'points': 2},
        'has_many_reviews': {'condition': lambda df: df['pssstore_stars_rating_count'] > 100, 'points': 2},
        
        'has_publisher_info': {'condition': lambda df: df['publisher'].notna(), 'points': 1},  # Ajuster dès que liste majors
        'has_dlcs': {'condition': lambda df: df['dlcs_count'] > 0, 'points': 1},
        'has_completion_info': {'condition': lambda df: df['hours_completionist'].notna(), 'points': 1},
        
        # hours main campaign
        'has_difficulty_info': {'condition': lambda df: df['difficulty'].notna(), 'points': 1},
        'has_deluxe_pack': {'condition': lambda df: df['packs_deluxe_count'] > 0, 'points': 1},
        'has_voice_languages': {'condition': lambda df: df['voice_languages'].notna(), 'points': 1},
        'has_subtitle_languages': {'condition': lambda df: df['subtitle_languages'].notna(), 'points': 1},
    }
    
    score = pd.Series(0, index=df.index, name='visibility_score')
    
    for rule_name, rule_config in VISIBILITY_SCORE_RULES.items():
        mask = rule_config['condition'](df)
        points = rule_config['points']
        count = mask.sum()
        score[mask] += points
    
    return score

In [493]:
df_visibility_score['visibility_score'] = calculate_visibility_score(df_visibility_score)

In [494]:
# Statistiques du score
print(f" Distribution du score:")
print(f"   Min: {df_visibility_score['visibility_score'].min()}")
print(f"   Max: {df_visibility_score['visibility_score'].max()}")
print(f"   Moyenne: {df_visibility_score['visibility_score'].mean():.2f}")
print(f"   Médiane: {df_visibility_score['visibility_score'].median():.2f}")
print(f"   Écart-type: {df_visibility_score['visibility_score'].std():.2f}")

 Distribution du score:
   Min: -2
   Max: 11
   Moyenne: 4.50
   Médiane: 4.00
   Écart-type: 3.23


In [495]:
# Catégoriser
df_visibility_score['visibility_category'] = pd.cut(
    df_visibility_score['visibility_score'],
    bins=[-float('inf'), 2, 4, 7, float('inf')],
    labels=['obscure', 'low_profile', 'known', 'high_profile']
)

In [496]:
# Analyse par catégorie
print("Statistiques par catégorie:")
for cat in ['obscure', 'low_profile', 'known', 'high_profile']:
    subset = df_visibility_score[df_visibility_score['visibility_category'] == cat]
    if len(subset) > 0:
        avg_price = subset['base_price'].mean()
        avg_rating = subset['visibility_score'].mean()
        
        print(f"\n   {cat:15s} ({len(subset):4d} jeux):")
        print(f"      Prix moyen: €{avg_price:.2f}")
        print(f"      Note moyenne: {avg_rating:.2f}")

Statistiques par catégorie:

   obscure         (1604 jeux):
      Prix moyen: €8.52
      Note moyenne: 0.85

   low_profile     (1339 jeux):
      Prix moyen: €12.80
      Note moyenne: 3.46

   known           (1347 jeux):
      Prix moyen: €19.49
      Note moyenne: 5.90

   high_profile    (1092 jeux):
      Prix moyen: €39.06
      Note moyenne: 9.40


In [497]:
# merge colonne visibility cat
df_featured = pd.concat([df_featured, df_visibility_score[['visibility_category']]], axis=1)

In [498]:
# Pour aller plus loin score de visibilité

# Version avec pondération dynamique: Version avec pondération basée sur l'importance réelle des features (à ajuster après analyse de corrélation)

## Feature combiné contenu exclusif playstation

In [499]:
df_exclusif_playstation_content = df_featured.copy()

In [500]:
# playstaion_exclu_content (is_ps_exclusive or is_opti_ps5_pro or is_vr)

def create_exclusif_playstation_content_feature(df:pd.DataFrame):
    df['exclusif_playstation_content'] = (
        (df['is_ps_exclusive'] == 1) | 
        (df['is_opti_ps5_pro'] == 1) | 
        (df['is_vr'] == 1)
    ).astype(int)
    
    return df

In [501]:
df_exclusif_playstation_content = create_exclusif_playstation_content_feature(df_exclusif_playstation_content)

In [502]:
print("Distribution contenu exclusif playstation:")
print("=" * 60)
total_assignments = df_exclusif_playstation_content['exclusif_playstation_content']
count_yes = (total_assignments == 1).sum()
pct_yes = count_yes / len(df_exclusif_playstation_content) * 100
print(f"   Oui: {count_yes:5d} jeux ({pct_yes:5.1f}%)")

count_yes = (total_assignments == 0).sum()
pct_yes = count_yes / len(df_exclusif_playstation_content) * 100
print(f"   Non: {count_yes:5d} jeux ({pct_yes:5.1f}%)")
    

Distribution contenu exclusif playstation:
   Oui:   480 jeux (  8.9%)
   Non:  4902 jeux ( 91.1%)


In [503]:
df_featured = pd.concat([df_featured, df_exclusif_playstation_content[['exclusif_playstation_content']]], axis=1)

## Combiner esrb et pegi

In [504]:
df_pegi_unified = df_featured.copy()

In [505]:
print(df_pegi_unified['esrb_rating'].unique())

['Everyone' nan 'Everyone 10+' 'Teen' 'Mature 17+' 'Adults Only 18+']


In [506]:
print(df_pegi_unified['pegi_rating'].unique())

<IntegerArray>
[7, 3, <NA>, 16, 12, 18]
Length: 6, dtype: Int64


In [507]:
# Combiner les colonnes esrb_rating et pegi_rating en une seule colonne 'esrb_pegi_merged'

# Mapping ESRB → PEGI
esrb_to_pegi = {
    'Everyone': 3,
    'Everyone 10+': 7,
    'Teen': 12,
    'Mature 17+': 16,
    'Adults Only 18+': 18
}

In [508]:
df_pegi_unified['pegi_unified'] = df_pegi_unified['pegi_rating'].fillna(df_pegi_unified['esrb_rating'].map(esrb_to_pegi))

In [516]:
print("Distribution pegi:")
print("=" * 60)

total_assignments = df_pegi_unified['pegi_unified']

for esrb_index,pegi_index in esrb_to_pegi.items():
    count = (total_assignments == pegi_index).sum()
    pct = count / len(df_pegi_unified) * 100
    print(f"   {pegi_index}: {count:5d} jeux ({pct:5.1f}%)")

nan_count = df_pegi_unified['pegi_unified'].isna().sum()
nan_pct = (nan_count / len(df)) * 100
print(f"NaN: {nan_count} ({nan_pct:.2f}%)")

Distribution pegi:
   3:  2015 jeux ( 37.4%)
   7:  1064 jeux ( 19.8%)
   12:  1062 jeux ( 19.7%)
   16:   924 jeux ( 17.2%)
   18:   317 jeux (  5.9%)
NaN: 0 (0.00%)


In [510]:
df_featured = pd.concat([df_featured, df_pegi_unified[['pegi_unified']]], axis=1)

## Colonne bonus infos à sortie + 60 jours

In [511]:
# Y y a t'il eu une première baisse de prix d'au moins 10% durant les 60 premiers jours après la sortie ?
# En terme métiers, il serait acceptable d'attendre les 1 ou 2 premieres mois avant d'affiner les prédictions

## Supprimer colonnes inutiles

In [512]:
col_to_delete = [
    'price_history',
    'genres',
    'pssstore_stars_rating_count',
    'pssstore_stars_rating',
    'metacritic_critic_score',
    'metacritic_user_score',
    'hours_main_story',
    'hours_completionist'
]

# Si uniquement ps5 supprimer isPs5, check nombre valeur 0 dans la colonne is_ps5
if int(df_featured['is_ps5'].value_counts().get(0, 0)) == 0: 
    col_to_delete.append('is_ps5')
    
print(col_to_delete)

['price_history', 'genres', 'pssstore_stars_rating_count', 'pssstore_stars_rating', 'metacritic_critic_score', 'metacritic_user_score', 'hours_main_story', 'hours_completionist', 'is_ps5']


## Synthèse des features pour la prédiction

In [ ]:
# X (viser environ 20 features)

# base_price
# released_year
# released_month
# pssstore_stars_rating_count (à pondéré)
# is_indie
# has_microtransaction
# dlc_count
# packs_deluxe_count
# serie_count
# visibility_category (score_jeu_est_connus_et_documenté)
# genres_grouped * 5
# playstation_exclusif_content
# esrb_pegi_merged
# - 17

# niveau qualité (classification: inconnue, mauvais, moyen, bon, très bon)
# score_contenu_riche x price
# publisher_studio_importance -> A faire

# # !!pondéré nombre d'étoile avec ancienneté du jeu!!

# bonus aide prediction: jeu à baisser d'au moins 5 % dans les 30 premiers jours
# bonus aide prediction: jeu à baisser d'au moins 5 % dans les 60 premiers jours

# en attente:
# base_price_categorie (train / test distribution equilibré)

# Y
# 
# Regression nombre de jours
# 
# days_to_10_percent_discount
# days_to_20_percent_discount
# days_to_30_percent_discount
# days_to_40_percent_discount
# days_to_50_percent_discount

# Classification tranche délais avant promotion
# 
# days_to_10_percent_discount_category
# days_to_20_percent_discount_category
# days_to_30_percent_discount_category
# days_to_40_percent_discount_category
# days_to_50_percent_discount_category


# for col in col_to_delete:
#     df_interactions.drop([col], axis='columns', inplace=True)

## Exporter features et target basic

## Exporter features avancés

In [514]:
column_summary(df_featured)

Résumé détaillé des colonnes:
                              Column     Type  Non-Null Count  Null Count  Unique Values
                           game_name   object            5382           0           5382
                            id_store   object            5382           0           5382
                      short_url_name   object            5382           0           5382
                           publisher   object            5382           0           1083
                           developer   object            3920        1462           2709
                        release_date   object            5382           0           1347
               pssstore_stars_rating  float64            5382           0            352
         pssstore_stars_rating_count    int64            5382           0           1583
             metacritic_critic_score    Int64            1268        4114             67
         metacritic_critic_userscore    Int64            1268        4114       

In [515]:
df_featured.head(5)

,game_name,id_store,short_url_name,publisher,developer,release_date,pssstore_stars_rating,pssstore_stars_rating_count,metacritic_critic_score,metacritic_critic_userscore,...,days_to_75_percent_discount,days_to_75_percent_discount_category,genre_action_aventure,genre_roles,genre_sports,genre_reflexion,genre_rapide,visibility_category,exclusif_playstation_content,pegi_unified
0,"#Halloween, Super Puzzles Dream",EP8311-PPSA19174_00-0421646910657705,game-halloween-super-puzzles-dream-ps5,e-llusiontertainment,NaN,2023-10-30,1.57,14,<NA>,<NA>,...,NaN,jamais_ou_plus,0,0,0,1,0,obscure,0,7
1,#SinucaAttack,EP2005-PPSA06055_00-SINUCA0000000000,game-sinucaattack-ps5,eastasiasoft,NaN,2022-05-19,3.26,72,<NA>,<NA>,...,NaN,jamais_ou_plus,1,0,0,1,0,obscure,0,3
2,"#Wish travel, super puzzle dreams",EP8311-PPSA16513_00-0233078860249892,game-wish-travel-super-puzzle-dreams-ps5,e-llusiontertainment,NaN,2023-05-31,1.55,11,<NA>,<NA>,...,NaN,jamais_ou_plus,0,0,0,1,0,obscure,0,3
3,"#WomenUp, Super Puzzles Dream",EP8311-PPSA13840_00-0277389480637871,game-womenup-super-puzzles-dream-ps5,e-llusiontertainment,NaN,2023-01-26,1.44,18,<NA>,<NA>,...,NaN,jamais_ou_plus,0,0,0,1,1,obscure,0,3
4,"#Xmas, Super Puzzles Dream",EP8311-PPSA12662_00-0212989199890961,game-xmas-super-puzzles-dream-ps5,e-llusiontertainment,NaN,2022-12-22,1.17,23,<NA>,<NA>,...,NaN,jamais_ou_plus,0,0,0,1,0,obscure,0,3
